## Recalling the last time I studied this

Let me explain if you didn't read the project proposal. Several years before taking this AI/ML course, I performed a study to explain why electricity demand varies by day. It's a very basic question that if one can't answer then one doesn't have any business explaining anything else that goes on in electricity markets. I inherited the study using Heating Degree Days and Cooling Degree Days, and reimplemented it with a few more predictors like day of week and holidays.

I used Excel's multiple linear regression on just a few summarized raw features, and it worked pretty well. I thought it was hot stuff, but my boss at the time, with a Ph.D. in Econometrics, said "don't show this to anybody before seeing me". Unfortunate events intevened and I was unable to ever have her review it, so I still don't know what errors she saw, but here I'll try to recreate it here **making the same mistakes**. 

In [1]:
import pandas as pd
import numpy as np
target_df = pd.read_pickle("dataframes/target_df.pickle.gz", compression="infer")

In [2]:
target_df = target_df[['sum_spp_load', 'is_Thursday', 'is_Saturday', 'is_Sunday', 'is_holiday', 'SUM_CDD', 'SUM_HDD' ]]
target_df


KeyError: "['SUM_HDD', 'SUM_CDD'] not in index"

In [ ]:
## separate independent and dependent variables


In [ ]:
# Referencing https://www.analyticsvidhya.com/blog/2021/05/multiple-linear-regression-using-python-and-scikit-learn/

# separate the target, or dependent, attribute from the predicting attributes
X = target_df.drop('sum_spp_load',axis=1)
# separate the target attribute into Y for model training 
y = target_df['sum_spp_load']

In [ ]:
# from https://towardsdatascience.com/simple-and-multiple-linear-regression-in-python-c928425168f9


In [ ]:
import statsmodels.api as sm # import statsmodels 

#X = df["RM"] ## X usually means our input variables (or independent variables)
#y = target["MEDV"] ## Y usually means our output/dependent variable
X = sm.add_constant(X) ## let's add an intercept (beta_0) to our model

# Note the difference in argument order
model = sm.OLS(y, X).fit() ## sm.OLS(output, input)
predictions = model.predict(X)

# Print out the statistics
model.summary()

## I got better results by adding a sequence number as a predictor, because historically electric demand has grown over time.
The sequence number can cause a slightly better fit on growth over time. However, there is nothing that suggests that growth is linear.  

## How can I show that the sequence number is showing demand growth over time but isn't simply tying together similar days?

In [ ]:
X = target_df.drop('sum_spp_load',axis=1)
X = sm.add_constant(X) ## let's add an intercept (beta_0) to our model
X['sequence_num'] = X.index

# Note the difference in argument order
model = sm.OLS(y, X).fit() ## sm.OLS(output, input)
predictions = model.predict(X)

# Print out the statistics
model.summary()

## TODO: elaborate on the warnings above. To the best of my memory, these numbers fairly well match my previous results. 

## TODO:  plot modeled vs. actual. 

## What if I used EVERYTHING in my target DF all at once?
I know, curse of dimensionality, strong colinearity, etc.  Not advised. But what does the model do? 

In [ ]:
# what if I used EVERYTHING in my original DF? 
target_df = pd.read_pickle("dataframes/target_df.pickle.gz", compression="infer").dropna()
y = target_df['sum_spp_load']
X = target_df.drop(['sum_spp_load', 'opday'],axis=1)
X = sm.add_constant(X) ## let's add an intercept (beta_0) to our model
X['sequence_num'] = X.index

model = sm.OLS(y, X).fit() ## sm.OLS(output, input)
predictions = model.predict(X)

# Print out the statistics
model.summary()

## Todo:  save predictions for all points; compare later to ML models


## What if I normalize the predictors and target first? 


In [3]:
# what if I used EVERYTHING in my original DF? 
target_df = pd.read_pickle("dataframes/target_df.pickle.gz", compression="infer").dropna()
y = target_df['sum_spp_load']
X = target_df.drop(['sum_spp_load', 'opday'],axis=1)
X['sequence_num'] = X.index

# normalize predictor variables
X=(X-X.min())/(X.max()-X.min() + 0.0001)  # small constant to prevent div/0

X = sm.add_constant(X) ## let's add an intercept (beta_0) to our model

# normalize target variable
y=(y-y.min())/(y.max()-y.min())

model = sm.OLS(y, X).fit() ## sm.OLS(output, input)
predictions = model.predict(X)

# Print out the statistics
model.summary()

NameError: name 'sm' is not defined

## Exactly the same results! 

OLS must be robust to normalization problems.  I can however now see the relative impact of each predictor variable by its coefficient magnitude.

In [ ]:
## Next steps:  break out scikit-learn, and see what we can do with dimensionality reduction and modern altorithms.